In [1]:
import pyarrow as pa
import psutil
process = psutil.Process()

arrow_file_str = "/home/daniel/asim_workspace/data/nuplan_mini_val/2021.06.07.12.54.00_veh-35_01843_02314.arrow"
print(process.memory_info().rss // (1024 * 1024), "MB") 
with pa.memory_map(arrow_file_str, "rb") as source:
    loaded_array = pa.ipc.open_file(source).read_all()
print(process.memory_info().rss // (1024 * 1024), "MB") 

105 MB
107 MB


In [2]:
print(process.memory_info().rss // (1024 * 1024), "MB") 
df = loaded_array[0].to_numpy()
print(process.memory_info().rss // (1024 * 1024), "MB") 
del df
print(process.memory_info().rss // (1024 * 1024), "MB") 

107 MB
146 MB
146 MB


In [3]:
loaded_array

pyarrow.Table
timestamp: int64
detections_state: list<item: fixed_size_list<item: double>[9]>
  child 0, item: fixed_size_list<item: double>[9]
      child 0, item: double
detections_token: list<item: string>
  child 0, item: string
detections_type: list<item: int16>
  child 0, item: int16
ego_states: fixed_size_list<item: double>[18]
  child 0, item: double
traffic_light_ids: list<item: int64>
  child 0, item: int64
traffic_light_types: list<item: int16>
  child 0, item: int16
scenario_tag: list<item: string>
  child 0, item: string
----
timestamp: [[1623086749200571,1623086749300508,1623086749400460,1623086749500414,1623086749600361,...,1623087219701159,1623087219801174,1623087219901182,1623087220001182,1623087220101172]]
detections_state: [[[[664461.1027193672,3997785.9783366807,617.1165975590507,0,0,1.6068943075772588,4.1869940757751465,1.859743356704712,1.8062634468078613],[664454.2894697834,3997793.2223469988,617.2127621706597,0,0,1.5865052216855613,4.781141757965088,2.0139472484

In [4]:
my_metadata = {"traffic_light_types": "this is a test", "child": "this is another test"}

new_table = pa.table(loaded_array, schema=loaded_array.schema, metadata=my_metadata)
# new_table

ValueError: The 'names' and 'metadata' arguments are not valid when using Arrow PyCapsule Interface

In [ ]:
pylist = [{'n_legs': 2, 'animals': 'Flamingo'}, {'year': 2021, 'animals': 'Centipede'}]

my_metadata = {"traffic_light_types": "this is a test", "child": [12,3,4,5,]}
tab = pa.Table.from_pylist(pylist, metadata=my_metadata)

key = "child"

s = tab.schema.metadata.decode()


s

TypeError: expected bytes, list found

In [ ]:
s.keys()

dict_keys([b'traffic_light_types', b'child'])

In [ ]:
from dataclasses import dataclass, asdict
import json
import pyarrow as pa

@dataclass
class MyMetadata:
    name: str
    version: int
    tags: list[str]

# Serialize to metadata
metadata_obj = MyMetadata("model_v1", 42, ["production", "validated"])
metadata_json = json.dumps(asdict(metadata_obj))
print(metadata_json)

# Create table with metadata
table = pa.table({"col1": [1, 2, 3]})
table = table.replace_schema_metadata({"custom_metadata": metadata_json})

# Deserialize from metadata
recovered_dict = json.loads(table.schema.metadata[b"custom_metadata"].decode())
recovered_obj = MyMetadata(**recovered_dict)
print(recovered_obj)


{"name": "model_v1", "version": 42, "tags": ["production", "validated"]}
MyMetadata(name='model_v1', version=42, tags=['production', 'validated'])
